In [1]:
'''
## Automatic Test Generation

In this project, we will build an automatic test generation and grading platform!
All we have to do is to provide a topic, the number of questions and the number of options for each question!
Based on this information, a suitable test is generated, presented to the user and graded automatically!
'''
import openai
import os
import pandas as pd

In [5]:
'''
Remember to use the notebook as shown, you must set your OpenAI API Key as an environment variable. 
Obviously, there are many ways you could provide your API Key to the Python code, input() or even hard-coded, 
but those are typically not recommended for safety reasons. 
Having it as an environment variable let's the key live on the computer, but not actually be present in the code.

### Set-up Open AI API Key

We'll only need to do this once per computer
'''
#os.environ['OPENAI_API_KEY']='sk-<your key here>'
openai.api_key=os.getenv("OPENAI_API_KEY")

In [6]:
#Open AI Halucination
prompt="Give me details about technology startup called Subhasish Company"

In [7]:
respose=openai.Completion.create(engine='text-davinci-003',
                                prompt=prompt,
                                 max_tokens=256,
                                temperature=0.7)

print(respose['choices'][0]['text'])



Subhasish Company is a technology startup based in India, founded in 2017. The company focuses on developing innovative and disruptive technologies to help organizations in their digital transformation journey. The company has developed a range of products and solutions in the areas of Artificial Intelligence, Machine Learning, Internet of Things, Big Data Analytics, Cloud Computing and Cybersecurity. It has also developed a range of enterprise applications such as customer relationship management, enterprise resource planning and supply chain management. The company has a team of experienced professionals with expertise in the latest technologies and has been working on projects for leading organizations in the country.


In [8]:
#Open AI avoiding Halucination 
prompt="Give me details about technology startup called Subhasish Company. Only answer if you are 100% sure that this company exist, otherwise specify , 'i dont't know' "

In [9]:
respose=openai.Completion.create(engine='text-davinci-003',
                                prompt=prompt,
                                 max_tokens=256,
                                temperature=0.7)

print(respose['choices'][0]['text'])



I don't know.


In [38]:
'''
### Tell GPT how to generate the test

We tell GPT to create a multiple choiz quiz. Hence we define the topic, the number of 
possible answers as well as the number of questions.

To enable automatical grading later, GPT needs to incorporate the correct answer!
'''
def create_test_prompt(topic,num_questions,num_possible_answers):
    prompt=f"Create a Multiple choice quiz on the Topic of {topic} consisting of {num_questions} questions. "\
    + f"Each question should have {num_possible_answers} options. "\
    + f"Also include the correct answer for each question using the starting string 'Correct Answer:' "
    return prompt
    

In [39]:
create_test_prompt("Python",4,4)

"Create a Multiple choice quiz on the Topic of Python consisting of 4 questions. Each question should have 4 options. Also include the correct answer for each question using the starting string 'Correct Answer:' "

In [40]:
'''
### OpenAI API Call
Let's use text-davinci-003 for normal text generation
'''
responses=openai.Completion.create(engine='text-davinci-003',
                                prompt=create_test_prompt("Indian History",4,4),
                                 max_tokens=256,
                                temperature=0.7)

print(responses['choices'][0]['text'])



Q1. Who wrote the book "The Discovery of India"
A. Mahatma Gandhi
B. Jawaharlal Nehru
C. Subhash Chandra Bose
D. Raja Ram Mohan Roy

Correct Answer: B. Jawaharlal Nehru

Q2. Who was the first Prime Minister of India?
A. Subhash Chandra Bose
B. Mahatma Gandhi
C. Raja Ram Mohan Roy
D. Jawaharlal Nehru

Correct Answer: D. Jawaharlal Nehru

Q3. Who wrote the Indian National Anthem?
A. Mahatma Gandhi
B. Subhash Chandra Bose
C. Rabindranath Tagore
D. Jawaharlal Nehru

Correct Answer: C. Rabindranath Tagore

Q4. Who was the first President of India?
A. Raja Ram Mohan Roy
B. Subhash Chandra Bose
C. Rajendra Prasad
D. Jawaharlal Nehru

Correct Answer: C. Rajendra Prasad


In [41]:
responses['choices'][0]['text']

'\n\nQ1. Who wrote the book "The Discovery of India"\nA. Mahatma Gandhi\nB. Jawaharlal Nehru\nC. Subhash Chandra Bose\nD. Raja Ram Mohan Roy\n\nCorrect Answer: B. Jawaharlal Nehru\n\nQ2. Who was the first Prime Minister of India?\nA. Subhash Chandra Bose\nB. Mahatma Gandhi\nC. Raja Ram Mohan Roy\nD. Jawaharlal Nehru\n\nCorrect Answer: D. Jawaharlal Nehru\n\nQ3. Who wrote the Indian National Anthem?\nA. Mahatma Gandhi\nB. Subhash Chandra Bose\nC. Rabindranath Tagore\nD. Jawaharlal Nehru\n\nCorrect Answer: C. Rabindranath Tagore\n\nQ4. Who was the first President of India?\nA. Raja Ram Mohan Roy\nB. Subhash Chandra Bose\nC. Rajendra Prasad\nD. Jawaharlal Nehru\n\nCorrect Answer: C. Rajendra Prasad'

In [34]:
'''
### Q/A Extraction

We now need to extract the questions and answers to present them to the students later
'''
def create_student_view(test, num_questions):
    student_view = {1 : ""}
    question_number = 1
    for line in test.split("\n"):
        if not line.startswith("Correct Answer:"):
            student_view[question_number] += line+"\n"
        else:

            if question_number < num_questions:
                question_number+=1
                student_view[question_number] = ""
    return student_view

In [42]:
create_student_view(responses["choices"][0]["text"], 4)

{1: '\n\nQ1. Who wrote the book "The Discovery of India"\nA. Mahatma Gandhi\nB. Jawaharlal Nehru\nC. Subhash Chandra Bose\nD. Raja Ram Mohan Roy\n\n',
 2: '\nQ2. Who was the first Prime Minister of India?\nA. Subhash Chandra Bose\nB. Mahatma Gandhi\nC. Raja Ram Mohan Roy\nD. Jawaharlal Nehru\n\n',
 3: '\nQ3. Who wrote the Indian National Anthem?\nA. Mahatma Gandhi\nB. Subhash Chandra Bose\nC. Rabindranath Tagore\nD. Jawaharlal Nehru\n\n',
 4: '\nQ4. Who was the first President of India?\nA. Raja Ram Mohan Roy\nB. Subhash Chandra Bose\nC. Rajendra Prasad\nD. Jawaharlal Nehru\n\n'}

In [43]:
result=create_student_view(responses["choices"][0]["text"], 4)

In [44]:
for key in result:
    print(result[key])



Q1. Who wrote the book "The Discovery of India"
A. Mahatma Gandhi
B. Jawaharlal Nehru
C. Subhash Chandra Bose
D. Raja Ram Mohan Roy



Q2. Who was the first Prime Minister of India?
A. Subhash Chandra Bose
B. Mahatma Gandhi
C. Raja Ram Mohan Roy
D. Jawaharlal Nehru



Q3. Who wrote the Indian National Anthem?
A. Mahatma Gandhi
B. Subhash Chandra Bose
C. Rabindranath Tagore
D. Jawaharlal Nehru



Q4. Who was the first President of India?
A. Raja Ram Mohan Roy
B. Subhash Chandra Bose
C. Rajendra Prasad
D. Jawaharlal Nehru




In [45]:
#Extract answers from test
def extract_answers(test, num_questions):
    answers = {1 : ""}
    question_number = 1
    for line in test.split("\n"):
        if line.startswith("Correct Answer:"):
            answers[question_number] += line+"\n"

            if question_number < num_questions:
                question_number+=1
                answers[question_number] = ""
    return answers

In [51]:
extract_answers(responses["choices"][0]["text"], 4)

{1: 'Correct Answer: B. Jawaharlal Nehru\n',
 2: 'Correct Answer: D. Jawaharlal Nehru\n',
 3: 'Correct Answer: C. Rabindranath Tagore\n',
 4: 'Correct Answer: C. Rajendra Prasad\n'}

In [48]:
answers=extract_answers(responses["choices"][0]["text"], 4)

In [50]:
for key in answers:
    print(answers[key])

Correct Answer: B. Jawaharlal Nehru

Correct Answer: D. Jawaharlal Nehru

Correct Answer: C. Rabindranath Tagore

Correct Answer: C. Rajendra Prasad



In [53]:
student_view=create_student_view(responses["choices"][0]["text"], 4)
answers=extract_answers(responses["choices"][0]["text"], 4)

In [54]:
'''
### Exam simulation
Based on the extracted questions, we can now simulate the exam
'''
def takeExam(student_view):
    answers = {}
    for question, question_view in student_view.items():
        print(question_view)
        answer = input("Enter your answer: ")
        answers[question] = answer
    return answers

In [55]:
student_answers = takeExam(create_student_view(responses["choices"][0]["text"], 4))



Q1. Who wrote the book "The Discovery of India"
A. Mahatma Gandhi
B. Jawaharlal Nehru
C. Subhash Chandra Bose
D. Raja Ram Mohan Roy



Q2. Who was the first Prime Minister of India?
A. Subhash Chandra Bose
B. Mahatma Gandhi
C. Raja Ram Mohan Roy
D. Jawaharlal Nehru



Q3. Who wrote the Indian National Anthem?
A. Mahatma Gandhi
B. Subhash Chandra Bose
C. Rabindranath Tagore
D. Jawaharlal Nehru



Q4. Who was the first President of India?
A. Raja Ram Mohan Roy
B. Subhash Chandra Bose
C. Rajendra Prasad
D. Jawaharlal Nehru




In [56]:
student_answers

{1: 'B', 2: 'D', 3: 'C', 4: 'C'}

In [57]:
'''
### Automatic Grading
Based on the student's answers and correct answers, we can now grade the test!
'''
def grade(correct_answer_dict, answers):
    correct_answers = 0
    for question, answer in answers.items():
        if answer.upper() == correct_answer_dict[question].upper()[16]:
            correct_answers+=1
    grade = 100 * correct_answers / len(answers)

    if grade < 60:
        passed = "Not passed!"
    else:
        passed = "Passed!"
    return f"{correct_answers} out of {len(answers)} correct! You achieved: {grade} % : {passed}"

In [58]:
grade(extract_answers(responses["choices"][0]["text"], 4), student_answers)

'4 out of 4 correct! You achieved: 100.0 % : Passed!'